In [1]:
import pandas as pd
import numpy as np
import copy

In [2]:
#This program deals with data that distinguish between empty cell and zero value in representations.
#However The program may also need to compute results based on cells that can be empty.
#The issue in this case is that operations on empty cells (NaN) generate NaN.
#We will create a function that would check the value of any cell before computation only.
#If the cell value is NaN it returns 0 to be used in computations, otherwise it returns the original value.
def X(i):
    return 0 if np.isnan(float(i)) else i

In [3]:
#reading input file for part1
df1 = pd.read_excel('Bplan-python2.xlsx', skiprows=range(0,1), header=None, nrows=52-1, usecols='B:Q')

#update cell values with direct descriptions
df1.loc[0,1]='Description'
df1.loc[1,1]='Time'
df1.loc[50,1]='Total Income'

#create column header to be the year values
head= {i+1:v for i,v in enumerate(df1[df1[1]=='Year'].values.flatten().tolist())}
head[1]='L2'
df1.rename(columns=head,inplace=True)

#delete rows that are all empty from the selected part of file
df1.drop(index=[2,3,6,8,9,49],inplace=True)
df1.reset_index(inplace=True)
df1.drop(columns=['index'],inplace=True)

#create parent index
parent_index=['Input','Companies share by volume', 'Number of companies', 'Discounts per company ',
              'Fee per year per company type', 'Income per company type']
child1='Input'
for i in range(df1.shape[0]):
    child = df1.loc[i,'L2']
    if child in parent_index:
        child1 = child
    df1.loc[i,'L1']=child1

#drop rows that have duplicated parent and child indices.
df1.drop(df1.loc[df1['L2'].isin(parent_index)].index, inplace=True)

#create multi_index dataframe
df1.set_index(['L1','L2'],inplace=True)

df1.loc[[('Input','Time')],2021]=0
df1.loc[[('Input','No. Users EU')],2021]=0

C:\ProgramData\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [4]:
#read input file for part2
df2 = pd.read_excel('Bplan-python2.xlsx', skiprows=range(0,4), header=None, nrows=13-4, usecols='A:B')

#drop empty values
df2.dropna(subset=[0],inplace=True)

#create dictionary of constants to be used for calculations
consts = dict(zip(df2[1],df2[0]))

#convert the constants_dict to dataframe suitable for HTML usage
df2 = pd.DataFrame(list(consts.items()), columns=['Input Variable', 'Input Value'])
df2.set_index('Input Variable',inplace=True)

In [5]:
#calculations for part1
comp_type=['Global Corp','Multinational','Big','Mid','Small','Startups']

eu_const = consts['No. Users EU']
tca_const = consts['Total companies affiliated']

#the below multiplier is present in Bplan (ref: D13:Q13) but not included in VectoriaWebBplan
tca_mult = {2021:0, 2022:0.5, 2023:0.5, 2024:0.6, 2025:1.15, 2026:1, 2027:1, 2028:1, 2029:1, 2030:1, 2031:1, 2032:1, 2033:1, 2034:1, 2035:1}

#the below multiplier is present in Bplan (ref: D7:Q7) but not included in VectoriaWebBplan
eu_mult = {2021:0, 2022:0.1, 2023:0.4, 2024:0.7, 2025:1.2, 2026:1, 2027:1, 2028:1, 2029:1, 2030:1, 2031:1, 2032:1, 2033:1, 2034:1, 2035:1}

#create indices for looping
years=df1.columns.to_list()
time = {i:v for v,i in enumerate(years)}

#calculations
companies_affiliated = {i:0 for i in years}
total_companies_affiliated = {i:0 for i in years}
sigma_eq = {i:0 for i in years}
users_eu = {i:0 for i in years}
fee_user_corp = {i:0 for i in years}
tot_inc_no_disc = {i:0 for i in years}

for key,val in companies_affiliated.items():
    if key==df1.columns[0]:
        companies_affiliated[key]=np.nan
        total_companies_affiliated[key]=np.nan
        sigma_eq[key]=np.nan
        users_eu[key]=0
        fee_user_corp[key]=np.nan
        tot_inc_no_disc[key]=np.nan
        
    else:
        companies_affiliated[key] = (1/2) + np.sign((X(time[key])-1)/(X(time[years[-1]])-1) - consts['Mu (companies sigmoide)'])/2*(1-np.exp((-np.sqrt(2)/consts['Sigma (companies sigmoide)'])*np.absolute((X(time[key])-1)/(X(time[years[-1]])-1)-consts['Mu (companies sigmoide)'])))
        total_companies_affiliated[key]= X(companies_affiliated[key])*tca_const*tca_mult[key]
    
        sigma_eq[key] = (1/2) + np.sign((X(time[key])-1)/(X(time[years[-1]])-1) - consts['Mu (users sigmoide)'])/2*(1-np.exp((-np.sqrt(2)/consts['Sigma (users sigmoide)'])*np.absolute((X(time[key])-1)/(X(time[years[-1]])-1)-consts['Mu (users sigmoide)'])))
        users_eu[key]= X(sigma_eq[key])*eu_const*eu_mult[key]
        fee_user_corp[key]=X(users_eu[key])*consts['API Fees euros']
        
        tot_inc_no_disc[key] = total_companies_affiliated[key] * fee_user_corp[key]
        
    df1.loc[[('Input','Sigma (users sigmoide)')],key]=X(sigma_eq[key])
    df1.loc[[('Input','No. Users EU')],key]=X(users_eu[key])
    df1.loc[[('Input','Fee as per user base (per corp)')],key]=X(fee_user_corp[key])
    df1.loc[[('Input','Companies affialiated')],key]=X(companies_affiliated[key])
    df1.loc[[('Input','Total companies affiliated')],key]=X(total_companies_affiliated[key])
    df1.loc[[('Input','Total income no discount')],key]=X(tot_inc_no_disc[key])
    
for item in comp_type:
    for year in years:
        if year == 2021:
            df1.loc[[('Number of companies',item)],year]=np.nan #no instruction given in Bplan to update 2021
            df1.loc[[('Fee per year per company type',item)],year]=np.nan #no instruction given in Bplan to update 2021
            df1.loc[[('Income per company type',item)],year]=np.nan #no instruction given in Bplan to update 2021
        elif year == 2022:
            df1.loc[[('Number of companies',item)],year]=(X(total_companies_affiliated[year])*X(float(df1.loc[[('Companies share by volume',item)],year].values)))
            df1.loc[[('Fee per year per company type',item)],year]=(X(fee_user_corp[year])*float(1-X(df1.loc[[('Discounts per company ',item)],year].values)))
            #the below formula is the same as that in Bplan but generates slightly different result, probably because of rounds
            df1.loc[[('Income per company type',item)],year]=(float(X(fee_user_corp[year])*float(X(df1.loc[[('Companies share by volume',item)],year].values))*X(total_companies_affiliated[year])-(X(fee_user_corp[year])*float(X(df1.loc[[('Companies share by volume',item)],year].values))*X(total_companies_affiliated[year])*X(df1.loc[[('Discounts per company ',item)],year].values))))
               
        else:
            df1.loc[[('Number of companies',item)],year]=(X(total_companies_affiliated[year])*float(X(df1.loc[[('Companies share by volume',item)],year].values)))
            df1.loc[[('Fee per year per company type',item)],year]=(X(fee_user_corp[year])*float(1-X(df1.loc[[('Discounts per company ',item)],year].values)))
            #the below formula is the same as that in Bplan but generates slightly different result, probably because of rounds
            df1.loc[[('Income per company type',item)],year]=(float(X(fee_user_corp[year])*float(X(df1.loc[[('Companies share by volume',item)],year].values))*X(total_companies_affiliated[year])-(X(fee_user_corp[year])*float(X(df1.loc[[('Companies share by volume',item)],year].values))*X(total_companies_affiliated[year])*X(df1.loc[[('Discounts per company ',item)],year].values))))
    
df1.loc[[('Number of companies','Total')],:]=df1.loc[[('Number of companies')]][:-1].sum().values #Update number of companies total
df1.loc[[('Income per company type','Total Income')],:]=df1.loc[[('Income per company type')]][:-1].sum().values #Update income total

In [6]:
#reading input file for part3
df3 = pd.read_excel('Bplan-python2.xlsx', skiprows=range(0,54), header=None, nrows=56-54, usecols='A:Q')

#update cell values with direct descriptions
df3.loc[:,0]='TOTAL INCOME Data Access'
df3.loc[0,1]='Yearly'
df3.loc[1,1]='ACUM'

#create column headers and set multi_index
head= {i+2:v for i,v in enumerate(df1.columns.tolist())}
head[0]='L1'
head[1]='L2'
df3.rename(columns=head,inplace=True)
df3.set_index(['L1','L2'],inplace=True)

#calculations for part3
for year in years:
    if year==2021:
        df3.loc[[('TOTAL INCOME Data Access','Yearly')],year]=0
        df3.loc[[('TOTAL INCOME Data Access','ACUM')],year]=0
    else:
        df3.loc[[('TOTAL INCOME Data Access','Yearly')],year]=(float(X(df1.loc[[('Income per company type','Total Income')],year].values)))
        df3.loc[[('TOTAL INCOME Data Access','ACUM')],year]=(float(X(df3.loc[[('TOTAL INCOME Data Access','ACUM')],year-1].values))+float(X(df3.loc[[('TOTAL INCOME Data Access','Yearly')],year].values)))

In [7]:
#reading input file for part4
df4 = pd.read_excel('Bplan-python2.xlsx', skiprows=range(0,57), header=None, nrows=124-57, usecols='A:Q')

#create column headers
head= {i+2:v for i,v in enumerate(df1.columns.tolist())}
head[0]='L1'
head[1]='L2'
df4.rename(columns=head,inplace=True)

#no calculations for this part since it is all inputs.

In [8]:
#reading input file for part5
df5 = pd.read_excel('Bplan-python2.xlsx', skiprows=range(0,129), header=None, nrows=132-129, usecols='A:Q')

#create column headers
head= {i+2:v for i,v in enumerate(df1.columns.tolist())}
head[0]='L1'
head[1]='L2'
df5.rename(columns=head,inplace=True)
df5.drop(columns=['L2'],inplace=True)
df5.set_index(['L1'],inplace=True)

#calculations
for year in years:
    df5.loc['TOTAL INCOMES EUR',year]=(float(X(df3.loc[[('TOTAL INCOME Data Access','Yearly')],year].values)))
    df5.loc['TOTAL COSTS EUR',year]=(float(X(df4.loc[63,year])) + float(X(df4.loc[24,year])))
    df5.loc['MARGIN',year]= (float(X(df5.loc['TOTAL INCOMES EUR',year])) - float(X(df5.loc['TOTAL COSTS EUR',year])))

In [9]:
#reading input file for parts8,9 (parts 6,7 depend on part 8 as such will be done after)
df8 = pd.read_excel('Bplan-python2.xlsx', skiprows=range(0,162), header=0, nrows=163-162, usecols='C:F')
R=df8.columns[0]
df8.rename(columns={R:'R'},inplace=True)

row = pd.Series({'Input Value':pd.read_excel('Bplan-python2.xlsx', index_col=None, skiprows=range(0,127), usecols = "S", header = 1, nrows=0).columns.values[0]},name='Discount Rate')
df2=df2.append(row)
VNA = sum([df5.loc['TOTAL INCOMES EUR',2035]/((1+0.05)**(i+1)) for i in range(7)])
row = pd.Series({'Input Value':VNA}, name='VNA')
df2=df2.append(row)

iir = df8.T.to_dict()[0]

df9 = pd.read_excel('Bplan-python2.xlsx', skiprows=range(0,164), header=None, nrows=169-164, usecols='B:Q')
df9.loc[2,1]='AIR%'
df9.loc[0,1]='Seq'
head=['L1']
head.extend([x for x in range(2021,2036)])
head={i+1:v for i,v in enumerate(head)}
df9.rename(columns=head,inplace=True)
df9.set_index(['L1'],inplace=True)

power_dict={year:2035-year for year in years}

for year in years:
    if year ==2021:
        df9.loc['Adapting interest rate',year]=100 #no equation given in file
        df9.loc['Exchange rate (token/Euro)',year]=(1+iir['R'])**power_dict[year]
        df9.loc['Exchange rate (Euro/token)',year]=1/((1+iir['R'])**power_dict[year])
    else:
        df9.loc['Adapting interest rate',year]= (1/2) - np.sign((X(df9.loc['Seq',year])-1)/(X(df9.loc['Seq',years[-1]])-1) - iir['Mu'])/2*(1-np.exp((-np.sqrt(2)/iir['Sigma'])*np.absolute((X(df9.loc['Seq',year])-1)/(X(df9.loc['Seq',years[-1]])-1)-iir['Mu'])))
        df9.loc['AIR%',year]=X(df9.loc['Adapting interest rate',year])*X(df9.loc['Adapting interest rate',years[0]])
        df9.loc['Exchange rate (token/Euro)',year]=(1+iir['R'])**power_dict[year]
        df9.loc['Exchange rate (Euro/token)',year]=1/((1+iir['R'])**power_dict[year])

In [10]:
df10 = pd.read_excel('Bplan-python2.xlsx', skiprows=range(0,174), header=0, nrows=181-175, usecols='B:H')
df10.rename(columns={'Unnamed: 1':'L1'},inplace=True)

In [11]:
df6 = pd.read_excel('Bplan-python2.xlsx', skiprows=range(0,135), header=0, nrows=137-136, usecols='D:G')
df6['Total commitment share']=3
df6['Commitment A']=df6['Total commitment share']-df6['Commitment B']-df6['Commitment C']
token = df6.T.to_dict()[0]

In [12]:
df7 = pd.read_excel('Bplan-python2.xlsx', skiprows=range(0,137), header=None, nrows=158-137, usecols='B:Q')
df7.drop(index=18,inplace=True)
df7.rename(columns={1:'L1'},inplace=True)
df7.set_index(['L1'],inplace=True)
head= {i+2:v for i,v in enumerate(df1.columns.tolist())}
df7.rename(columns=head,inplace=True)

In [13]:
n_nodes_dict = {i:0 for i in years}
for year in years:
    n_nodes_dict[year]= float(df1.loc[[('Input','No. Users EU')],year].values)/10000
n_nodes_dict[2021]=0 #Number of nodes is a constant in file
n_nodes_dict[2022]=10 #Number of nodes is a constant in file
n_nodes_dict[2023]=15 #Number of nodes is a constant in file
n_nodes_dict[2024]=20 #Number of nodes is a constant in file

for year in years:
    if year == 2021:
        df7.loc['Total business unitsl costs EUR',year]= (float(X(df5.loc['TOTAL COSTS EUR',year]))) if X(df4.loc[63,year])>0 else 0
        df7.loc['Total business units costs PELA',year]= (float(X(df5.loc['TOTAL COSTS EUR',year]))*float(X(df9.loc['Exchange rate (token/Euro)',year])))
        df7.loc['Cost per node/year',year]=6000 #no equation given in file
        df7.loc['Number of nodes',year]=n_nodes_dict[year]
        df7.loc['Total Cost Corps Data crowdsourced EUR',year]= (float(X(df4.loc[66,year])))
        df7.loc['Total Tech Costs EUR',year]= np.nan #no equation given in file
        df7.loc['aux. S',year]=np.nan #no equation given in file
        df7.loc['Total cost mining EUR',year]=X(df7.loc['Number of nodes',year])*X(df7.loc['Cost per node/year',years[0]])
        df7.loc['Total cost mining PELA',year]=(X(df7.loc['Total cost mining EUR',year])*float(X(df9.loc['Exchange rate (token/Euro)',years[1]])))
        df7.loc['Total Cost Corps Data crowdsourced PELA',year]=(X(df7.loc['Total Cost Corps Data crowdsourced EUR',year])*float(X(df9.loc['Exchange rate (token/Euro)',year])))
        df7.loc['Total Tech Costs PELA',year]=X(df7.loc['Total Tech Costs EUR',year])*X(float(df9.loc['Exchange rate (token/Euro)',year]))
        df7.loc['Total Contributions Costs in Token',year]=X(df7.loc['Total Tech Costs PELA',year])+X(df7.loc['Total cost mining PELA',year])+X(df7.loc['Total business units costs PELA',year])+X(df7.loc['Total Cost Corps Data crowdsourced PELA',year])
        df7.loc['ABX',year]=np.nan #no equation given in file
        df7.loc['CX',year]=np.nan #no equation given in file
             
    else:
        df7.loc['Total business unitsl costs EUR',year]= (float(X(df4.loc[63,year])))
        df7.loc['Total business units costs PELA',year]= (float(X(df4.loc[63,year]))*float(X(df9.loc['Exchange rate (token/Euro)',year])))
        df7.loc['Cost per node/year',year]=np.nan #no equation given in file
        df7.loc['Number of nodes',year]=n_nodes_dict[year]
        df7.loc['Total Cost Corps Data crowdsourced EUR',year]= (float(X(df4.loc[66,year])))
        df7.loc['Total Tech Costs EUR',year]= round(float(X(df4.loc[24,year])))
        df7.loc['aux. S',year]=(((1-token['Token decay'])**(float(X(df1.loc[[('Input','Time')],year].values))-1))*(float(X(df1.loc[[('Input','No. Users EU')],year].values))))
        df7.loc['Total cost mining EUR',year]=X(df7.loc['Number of nodes',year])*X(df7.loc['Cost per node/year',years[0]])
        df7.loc['Total cost mining PELA',year]=(X(df7.loc['Total cost mining EUR',year])*float(X(df9.loc['Exchange rate (token/Euro)',year])))
        df7.loc['Total Cost Corps Data crowdsourced PELA',year]=(X(df7.loc['Total Cost Corps Data crowdsourced EUR',year])*float(X(df9.loc['Exchange rate (token/Euro)',year])))
        df7.loc['Total Tech Costs PELA',year]=X(df7.loc['Total Tech Costs EUR',year])*float(X(df9.loc['Exchange rate (token/Euro)',year]))
        df7.loc['Total Contributions Costs in Token',year]=X(df7.loc['Total Tech Costs PELA',year])+X(df7.loc['Total cost mining PELA',year])+X(df7.loc['Total business units costs PELA',year])+X(df7.loc['Total Cost Corps Data crowdsourced PELA',year])
        if year == 2022:
            df7.loc['ABX',year]= (X(df1.loc[[('Input','No. Users EU')],year].values)*(token['Commitment A']+token['Commitment B'])/token['Commitment A'])
            df7.loc['CX',year]=token['Commitment C']/token['Commitment A']*X(df1.loc[[('Input','No. Users EU')],year].values)
        elif year == 2023:
            df7.loc['ABX',year]= (X(df1.loc[[('Input','No. Users EU')],year].values) - X(df1.loc[[('Input','No. Users EU')],year-1].values))*((1-token['Token decay'])**df1.loc[[('Input','Time')],year-1].values)*(1+(token['Commitment B']/token['Commitment A']))
            df7.loc['CX',year]=token['Commitment C']/token['Commitment A']*X(df1.loc[[('Input','No. Users EU')],year].values)*(1-token['Token decay'])**df1.loc[[('Input','Time')],year-1].values
        else:
            df7.loc['ABX',year]= (X(df1.loc[[('Input','No. Users EU')],year].values) - X(df1.loc[[('Input','No. Users EU')],year-1].values))*((1-token['Token decay'])**df1.loc[[('Input','Time')],year-1].values)*(token['Commitment A']+ token['Commitment B'])/token['Commitment A']
            df7.loc['CX',year]=token['Commitment C']/token['Commitment A']*X(df1.loc[[('Input','No. Users EU')],year].values)*(1-token['Token decay'])**df1.loc[[('Input','Time')],year-1].values
        
for year in years:
    if year == 2021:
        df7.loc['Citizen sign-up cost (referrral)',year]=np.nan #no equation given in file
        df7.loc['Sign in incentive',year]=np.nan #no equation given in file
        df7.loc["Proof of citizen's commitment",year]=np.nan #no equation given in file
        df7.loc['Total stimulus investment token',year]=np.nan #no equation given in file
        df7.loc['Initial supply (costs 2021-23)',year]=np.nan #no equation given in file
        
    elif year == 2022: 
        #D138
        R130=float(VNA)
        R153=float(df7.loc['Total Contributions Costs in Token'].sum())
        E137=token['Commitment A']
        F137=token['Commitment B']
        G137=token['Commitment C']
        R151=float(df7.loc['aux. S'][1:].sum())
        D137=token['Token decay']
        T151=float(df7.loc['aux. S'][1:-1].sum())
        D213=float(X(df7.loc['ABX'][1:].sum()))+float(X(df7.loc['CX'][1:].sum()))      
        
        df7.loc['Citizen sign-up cost (referrral)',year]=(R130-R153)/D213
        
        df7.loc['Sign in incentive',year]=X(df7.loc['Citizen sign-up cost (referrral)',year])*X(token['Commitment B'])/X(token['Commitment A'])
        df7.loc["Proof of citizen's commitment",year]=X(df7.loc['Citizen sign-up cost (referrral)',year])*X(token['Commitment C'])/X(token['Commitment A'])
        
        #D152
        D138=X(df7.loc['Citizen sign-up cost (referrral)',year])
        D139=X(df7.loc['Sign in incentive',year])
        D140=X(df7.loc["Proof of citizen's commitment",year])
        D7=(float(X(df1.loc[[('Input','No. Users EU')],year].values)))
        df7.loc['Total stimulus investment token',year]=((D138*D7)+(D139*D7)+(D140*D7))
        
        df7.loc['Initial supply (costs 2021-23)',year]=X(df7.loc['Total business units costs PELA',2021])+X(df7.loc['Total business units costs PELA',2022])+X(df7.loc['Total business units costs PELA',2023])
        
    else:
        df7.loc['Citizen sign-up cost (referrral)',year]=X(df7.loc['Citizen sign-up cost (referrral)',year-1])*(1-token['Token decay'])
        df7.loc['Sign in incentive',year]=X(df7.loc['Citizen sign-up cost (referrral)',year])*token['Commitment B']/X(token['Commitment A'])
        df7.loc["Proof of citizen's commitment",year]=X(df7.loc['Citizen sign-up cost (referrral)',year])*token['Commitment C']/X(token['Commitment A'])
        
        #E152
        E138=X(df7.loc['Citizen sign-up cost (referrral)',year])
        E139=X(df7.loc['Sign in incentive',year])
        E140=X(df7.loc["Proof of citizen's commitment",year])
        E7=(float(X(df1.loc[[('Input','No. Users EU')],year].values)))
        D7=(float(X(df1.loc[[('Input','No. Users EU')],year-1].values)))
        df7.loc['Total stimulus investment token',year]=((E138*(E7-D7)+(E139*(E7-D7)+(E140*E7))))
        
        df7.loc['Initial supply (costs 2021-23)',year]=np.nan #no equation given in file
    
    #C154
    C142=X(df7.loc['Total business units costs PELA',year])
    C150=X(df7.loc['Total Tech Costs PELA',year])
    C152=X(df7.loc['Total stimulus investment token',year])
    C146=X(df7.loc['Total cost mining PELA',year])
    C148=X(df7.loc['Total Cost Corps Data crowdsourced PELA',year])
    df7.loc['TOTAL TOKEN SUPPLY',year]=C142+C150+C152+C146+C148

for year in years:
    if year == 2021:
        df7.loc['TOTAL CIRCULATING TOKEN SUPPLY',year]=X(df7.loc['TOTAL TOKEN SUPPLY',year])-X(df7.loc['Vesting',year])
    else:
        df7.loc['TOTAL CIRCULATING TOKEN SUPPLY',year]=X(df7.loc['TOTAL TOKEN SUPPLY',year])-X(df7.loc['Vesting',year])+X(df7.loc['TOTAL CIRCULATING TOKEN SUPPLY',year-1])
    
df7.drop(index=['ABX','CX'],inplace=True)
       

In [14]:
#block1: df2,df6,df8
df02=pd.concat([df2,df6.T.rename(columns={0:'Input Value'}),df8.T.rename(columns={0:'Input Value'})])

#block2: df1
df01=copy.deepcopy(df1)
df01_idxs=df01.index.tolist()[:-1]
df01_idxs.extend([('Total Income', '')])
df01_idxs[0]=('Input','')
df01_idxs[1]=('Input','')
df01.index=pd.MultiIndex.from_tuples(df01_idxs,names=None)
df01.drop([('Input','Sigma (users sigmoide)'),('Input','Companies affialiated')], axis=0, inplace=True)
df01.drop(['Discounts per company ','Fee per year per company type','Income per company type'],level=0, axis=0, inplace=True)

#block3: df4 ==> df 3 remains same, need to remove L1/L2
df04 = copy.deepcopy(df4[df4['L1'].isin(['TOTAL TECH PLATFORM COSTS','TOTAL BUSINESS UNIT COSTS','TOTAL CORP DATA COSTS'])])
df04.set_index('L1',inplace=True)
df04.index.name=None
df04.loc['TOTAL BUSINESS UNIT COSTS','L2']=df04.loc['TOTAL BUSINESS UNIT COSTS',:][1:].sum()
df04.rename(columns={'L2':''},inplace=True)

#block4: df7 ==> df 5 remains same, need to remove L1/L2
idxs=['Citizen sign-up cost (referrral)', 'Sign in incentive',"Proof of citizen's commitment", 
      'Total business units costs PELA', 'Total cost mining PELA','Total Cost Corps Data crowdsourced PELA', 
      'Total Tech Costs PELA','Total stimulus investment token','Total Contributions Costs in Token','TOTAL TOKEN SUPPLY', 
      'Initial supply (costs 2021-23)']
mod_idxs=['Total referral costs (aux.)','Total sign in costs (aux.)','Total Data provision costs (aux)']
sym=['(A)','(B)','(C)','(D)','(E)','(F)','(G)','V=(A+B+C)','W=(D+E+F+G)','TS=(V+W)','D values (2021+2022+2023)']
s=[df7.loc[i,:].dropna().values.sum()/df7.loc['TOTAL TOKEN SUPPLY',:].dropna().values.sum() for i in idxs]
s[-1]=float(df7.loc['Initial supply (costs 2021-23)',2022])
y_prime=df1.iloc[[3],:].to_numpy()
for i in range(3):
    x=df7.iloc[[i],:].to_numpy()
    x=np.delete(x,0,1)
    if i!=2:
        y=y_prime[0,1:]-y_prime[0,:-1]
    else:
        y=np.delete(y_prime,0,1).T
    s[i]=float(np.matmul(x,y))/df7.loc['TOTAL TOKEN SUPPLY',:].dropna().values.sum()
    idxs[i]=mod_idxs[i]
df07=pd.DataFrame({'Value':s}, index=[idxs,sym])

#block5: df9
df09 = copy.deepcopy(df9.loc[['Exchange rate (token/Euro)','Exchange rate (Euro/token)']])
df09.index.name=None

#additional block: df3, df5
df03=copy.deepcopy(df3)
df05=copy.deepcopy(df5)
df03.index.names=[None,None]
df05.index.name=None

In [15]:
#create html file and write all dataframes to it.
with open("output.html", 'w') as f:
    f.write(
        df2.to_html(na_rep='',justify='center', 
                    float_format= lambda x:'{:,.0f}'.format(round(x)) if x>10 else'{:,.4f}'.format(x) if x<0.01 else '{:,.2f}'.format(x))+"<br><br>"
            + df1.to_html(na_rep='',justify='center', 
                          float_format= lambda x:'{:,.0f}'.format(round(x)) if x>10 else'{:,.2f}'.format(x))+"<br><br>"
            + df3.to_html(na_rep='',justify='center', 
                          float_format= lambda x:'{:,.0f}'.format(round(x)) if x>10 else'{:,.2f}'.format(x))+"<br><br>"
            + df4.to_html(na_rep='',justify='center', 
                          float_format= lambda x:'{:,.0f}'.format(round(x)) if x>10 else'{:,.2f}'.format(x))+"<br><br>"
            + df6.to_html(na_rep='',justify='center', 
                          float_format= lambda x:'{:,.0f}'.format(round(x)) if x>10 else'{:,.2f}'.format(x))+"<br><br>"
            + df7.to_html(na_rep='',justify='center', 
                          float_format= lambda x:'{:,.0f}'.format(round(x)) if x>10 else'{:,.2f}'.format(x))+"<br><br>"
            + df8.to_html(na_rep='',justify='center', 
                          float_format= lambda x:'{:,.0f}'.format(round(x)) if x>10 else'{:,.2f}'.format(x))+"<br><br>"
            + df9.to_html(na_rep='',justify='center', 
                          float_format= lambda x:'{:,.0f}'.format(round(x)) if x>10 else'{:,.2f}'.format(x))+"<br><br>"
            + df10.to_html(na_rep='',justify='center', 
                           float_format= lambda x:'{:,.0f}'.format(round(x)) if x>10 else'{:,.2f}'.format(x)))

In [16]:
#create html file and write all dataframes to it.
with open("output-short.html", 'w') as f:
    f.write(
        df02.to_html(na_rep='',justify='center', 
                     float_format= lambda x:'{:,.0f}'.format(round(x)) if x>10 else'{:,.4f}'.format(x) if x<0.01 else '{:,.2f}'.format(x))+"<br><br>"
        +df01.to_html(na_rep='',justify='center', 
                     float_format= lambda x:'{:,.0f}'.format(round(x)) if x>10 else'{:,.2f}'.format(x))+"<br><br>"
        +df03.to_html(na_rep='',justify='center', 
                     float_format= lambda x:'{:,.0f}'.format(round(x)) if x>10 else'{:,.2f}'.format(x))+"<br><br>"
        +df04.to_html(na_rep='',justify='center', 
                     float_format= lambda x:'{:,.0f}'.format(round(x)) if x>10 else'{:,.2f}'.format(x))+"<br><br>"
        +'<h2>TOKENOMICS</h2>'
        +df07.to_html(na_rep='',justify='center', 
                     float_format= lambda x:'{:,.0f}'.format(round(x)) if x>10 else'{:,.2%}'.format(x))+"<br><br>"
        +df09.to_html(na_rep='',justify='center', 
                     float_format= lambda x:'{:,.0f}'.format(round(x)) if x>10 else'{:,.2f}'.format(x)))